# Arboles de decision Bayesianos

El **Árbol de Decisión Bayesiano** es una extensión de los árboles de decisión tradicionales que incorpora principios de la estadística bayesiana para tomar decisiones en cada nodo. A diferencia de los árboles de decisión estándar, donde las decisiones se basan en divisiones determinísticas según una métrica como la ganancia de información o el índice de Gini, un árbol de decisión bayesiano asigna distribuciones de probabilidad a las características y calcula probabilidades condicionadas en cada nodo.

## Estructura del Modelo

1. **Nodos Internos**: En un árbol de decisión bayesiano, cada nodo interno representa una prueba o condición basada en una o más características del conjunto de datos. En lugar de realizar una división basada en un solo valor, se asigna una distribución de probabilidad a la característica relevante.

2. **Distribución de Probabilidad**: Cada característica en un nodo puede tener una distribución de probabilidad asociada. Por ejemplo, si una característica es continua, podría estar modelada con una distribución normal (Gaussiana). Si es categórica, podría estar modelada con una distribución multinomial.

3. **Hiperparámetros**: Los hiperparámetros son los parámetros que definen la distribución de probabilidad. En el caso de una distribución normal, los hiperparámetros serían la media y la varianza. Estos hiperparámetros pueden ser ajustados durante el entrenamiento del modelo.

4. **Probabilidades Condicionadas**: En cada nodo, se calculan las probabilidades condicionadas de las diferentes clases, dado el valor de la característica. Estas probabilidades se actualizan utilizando el teorema de Bayes a medida que se recorre el árbol.

5. **Clasificación Final**: Para clasificar una instancia, se recorre el árbol desde la raíz hasta una hoja, multiplicando las probabilidades a lo largo del camino. La clase con la mayor probabilidad final es asignada a la instancia.


### Ejemplo Matemático

Consideremos un ejemplo simple con una característica continua $x$ y dos posibles clases $C_1$ y $C_2$.

1. **Distribución de Probabilidad**: Supongamos que $x$ sigue una distribución normal:
   $
   P(x|C_i) = \frac{1}{\sqrt{2\pi \sigma_i^2}} \exp\left(-\frac{(x - \mu_i)^2}{2\sigma_i^2}\right)
   $
   donde $\mu_i$ y $\sigma_i^2$ son los hiperparámetros (media y varianza) para la clase $C_i$.

2. **Probabilidad a Priori**: Se asignan probabilidades a priori a las clases, $P(C_1)$ y $P(C_2)$.

3. **Probabilidad Posteriori**: Al observar un valor $x$, se calculan las probabilidades posteriores utilizando el teorema de Bayes:
   $
   P(C_i|x) = \frac{P(x|C_i) P(C_i)}{P(x)}
   $
   donde $P(x) = \sum_{i} P(x|C_i)P(C_i)$ es la probabilidad marginal de $x$.

4. **Decisión**: Se asigna la clase con la mayor probabilidad posterior:
   $
   \hat{C} = \arg\max_{i} P(C_i|x)
   $

## Ventajas y Desventajas

- **Ventajas**:
  - Permite incorporar incertidumbre en las decisiones.
  - Es más robusto ante la variabilidad en los datos.
  - Puede manejar datos con ruido y datos faltantes de manera más efectiva.

- **Desventajas**:
  - Mayor complejidad computacional debido al cálculo de probabilidades en cada nodo.
  - Requiere estimar y ajustar correctamente las distribuciones de probabilidad y sus hiperparámetros.

### Aplicaciones

Este modelo es útil en situaciones donde es importante capturar la incertidumbre en la toma de decisiones, como en sistemas de diagnóstico médico, detección de fraudes, y otras aplicaciones donde los riesgos asociados con decisiones incorrectas son altos.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sklearn.metrics import accuracy_score

# Simular datos
np.random.seed(42)
n_samples = 100

# Característica 1: Peso
X1_class0 = np.random.normal(160, 10, n_samples // 2)
X1_class1 = np.random.normal(180, 10, n_samples // 2)

# Característica 2: Tamaño
X2_class0 = np.random.normal(5, 1, n_samples // 2)
X2_class1 = np.random.normal(7, 1, n_samples // 2)

# Etiquetas
y_class0 = np.zeros(n_samples // 2)
y_class1 = np.ones(n_samples // 2)

# Unir datos
X = np.vstack((np.hstack((X1_class0, X1_class1)), np.hstack((X2_class0, X2_class1)))).T
y = np.hstack((y_class0, y_class1))

# Crear DataFrame
df = pd.DataFrame(X, columns=['Peso', 'Tamaño'])
df['Clase'] = y

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class BayesianDecisionTree:
    """
    Un árbol de decisión que utiliza un enfoque bayesiano para la clasificación.
    Este modelo combina la estructura de un árbol de decisión con la estimación
    de probabilidades basadas en distribuciones Gaussianas.
    """

    def __init__(self):
        """Inicializa el modelo como un diccionario vacío que contendrá la estructura del árbol."""
        self.model = {}

    def fit(self, X, y):
        """
        Ajusta el modelo a los datos de entrenamiento.

        Args:
            X (numpy.ndarray): Matriz de características de forma (n_samples, n_features).
            y (numpy.ndarray): Vector de etiquetas de clase de forma (n_samples,).

        """
        # La raíz del árbol se construye dividiendo los datos en nodos
        self.model['root'] = self._split_node(X, y)

    def _split_node(self, X, y):
        """
        Encuentra la mejor característica y umbral para dividir los datos en un nodo.

        Args:
            X (numpy.ndarray): Matriz de características.
            y (numpy.ndarray): Vector de etiquetas de clase.

        Returns:
            dict: Nodo dividido con la característica seleccionada, umbral y
                  las distribuciones de probabilidad condicional para cada rama (izquierda y derecha).
        """
        best_feature = None
        best_threshold = None
        best_score = float('inf')  # Inicializa el mejor puntaje con infinito

        # Itera sobre todas las características para encontrar la mejor
        for feature_idx in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_idx])  # Umbrales únicos de la característica
            for threshold in thresholds:
                score = self._calculate_score(X, y, feature_idx, threshold)
                if score < best_score:  # Busca minimizar el puntaje
                    best_score = score
                    best_feature = feature_idx
                    best_threshold = threshold

        # Divide los datos en dos subconjuntos: izquierdo y derecho
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        # Crea el nodo con la mejor característica y umbral encontrados
        left_node = {
            'feature': best_feature,
            'threshold': best_threshold,
            'left': self._calculate_likelihoods(X[left_indices], y[left_indices]),
            'right': self._calculate_likelihoods(X[right_indices], y[right_indices])
        }

        return left_node

    def _calculate_score(self, X, y, feature_idx, threshold):
        """
        Calcula un puntaje para una división específica basada en la suma de las probabilidades logarítmicas negativas.

        Args:
            X (numpy.ndarray): Matriz de características.
            y (numpy.ndarray): Vector de etiquetas de clase.
            feature_idx (int): Índice de la característica seleccionada.
            threshold (float): Umbral de la característica para la división.

        Returns:
            float: Puntaje basado en la suma de las probabilidades logarítmicas negativas.
        """
        # Divide los datos según el umbral
        left_indices = X[:, feature_idx] <= threshold
        right_indices = X[:, feature_idx] > threshold

        # Si una de las divisiones está vacía, se devuelve un puntaje infinito
        if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
            return float('inf')

        # Calcula las verosimilitudes para ambas ramas
        left_likelihood = self._calculate_likelihoods(X[left_indices], y[left_indices])
        right_likelihood = self._calculate_likelihoods(X[right_indices], y[right_indices])

        # El puntaje es la suma de las verosimilitudes negativas
        total_score = left_likelihood['score'] + right_likelihood['score']
        return total_score

    def _calculate_likelihoods(self, X, y):
        """
        Calcula la verosimilitud para cada clase dada una partición de los datos.

        Args:
            X (numpy.ndarray): Subconjunto de la matriz de características.
            y (numpy.ndarray): Subconjunto del vector de etiquetas de clase.

        Returns:
            dict: Verosimilitudes para cada clase, junto con el puntaje total de la partición.
        """
        classes = np.unique(y)
        likelihoods = {'score': 0}  # Inicializa el diccionario con 'score'

        for cls in classes:
            class_indices = (y == cls)
            mu = np.mean(X[class_indices], axis=0)  # Media de la característica
            sigma = np.std(X[class_indices], axis=0)  # Desviación estándar

            # Evita la división por cero
            sigma[sigma == 0] = 1e-4

            # Guarda los parámetros de la distribución gaussiana para la clase
            likelihoods[cls] = {
                'mu': mu,
                'sigma': sigma,
                'prior': np.mean(class_indices)  # Probabilidad a priori
            }

            # Calcula el puntaje como la suma negativa del logaritmo de la función de densidad
            likelihoods['score'] += -np.sum(np.log(norm.pdf(X[class_indices], mu, sigma) + 1e-6))

        return likelihoods

    def predict(self, X):
        """
        Predice las clases para un conjunto de muestras utilizando el árbol de decisión bayesiano.

        Args:
            X (numpy.ndarray): Matriz de características de las muestras a predecir.

        Returns:
            numpy.ndarray: Vector de predicciones para las muestras.
        """
        predictions = []
        for x in X:
            likelihoods = self.model['root']
            feature_idx = likelihoods['feature']
            threshold = likelihoods['threshold']

            # Decide si seguir a la izquierda o a la derecha en el árbol
            if x[feature_idx] <= threshold:
                probs = self._calculate_posterior(x, likelihoods['left'])
            else:
                probs = self._calculate_posterior(x, likelihoods['right'])

            # Predice la clase con la mayor probabilidad posterior
            predictions.append(max(probs, key=probs.get))

        return np.array(predictions)

    def _calculate_posterior(self, x, likelihoods):
        """
        Calcula las probabilidades posteriores para una muestra dada utilizando las verosimilitudes y las probabilidades a priori.

        Args:
            x (numpy.ndarray): Muestra individual.
            likelihoods (dict): Verosimilitudes y probabilidades a priori calculadas para una rama del árbol.

        Returns:
            dict: Probabilidades posteriores para cada clase.
        """
        posteriors = {}
        for cls, stats in likelihoods.items():
            if cls == 'score':  # Ignora 'score' en los cálculos
                continue
            prior = stats['prior']
            # Calcula la probabilidad del valor x dado la distribución gaussiana para la clase
            likelihood = np.prod(norm.pdf(x, stats['mu'], stats['sigma']) + 1e-6)
            # Multiplica por la probabilidad a priori para obtener la probabilidad posterior
            posteriors[cls] = prior * likelihood
        return posteriors


In [ ]:
# Inicializar y entrenar el modelo
model = BayesianDecisionTree()
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
